In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
image = cv2.imread('fig1.jpg')

In [ ]:
# 将图像从 BGR 转换为 RGB 格式
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 将图像转换为一维数组
pixels = image_rgb.reshape((-1, 3))

# 转换为float类型
pixels = np.float32(pixels)

# 定义停止条件
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)

# 设置聚类数量
k = 5

# 进行K均值聚类
_, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

# 将中心转换为8位无符号整数
centers = np.uint8(centers)

# 重新分配所有像素值
segmented_image = centers[labels.flatten()]

# 重新形状回图像的形状
segmented_image = segmented_image.reshape(image_rgb.shape)

# 显示原始图像和分割后的图像
plt.subplot(1, 2, 1)
plt.imshow(image_rgb)
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(segmented_image)
plt.title('Segmented Image')
plt.axis('off')

plt.show()

# 绘制颜色直方图
histogram = cv2.calcHist([segmented_image], [0], None, [k], [0, 256])
colors = centers
#plt.bar(range(len(histogram)), histogram, color=colors)
plt.title('Color Histogram')
plt.xlabel('Color')
plt.ylabel('Frequency')
plt.show()

In [ ]:
import cv2
import numpy as np

def extract_green_color_blocks(image, threshold=30):
    # 定义绿色范围（在 BGR 空间中）
    lower_green = np.array([76,125,49])
    upper_green = np.array([198,247,169])

    # 创建一个掩码，其中绿色范围内的像素为白色（255），其他像素为黑色（0）
    mask = cv2.inRange(image, lower_green, upper_green)
    
    # 对掩码进行形态学处理，以减少噪声并平滑轮廓
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # 在原始图像上应用掩码，提取出绿色的主要色块
    green_blocks = cv2.bitwise_and(image, image, mask=mask)

    # 查找色块的轮廓
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 绘制轮廓并将其加入到主要色块图像中
    cv2.drawContours(green_blocks, contours, -1, (0, 255, 0), 2)

    return green_blocks

# 读取图片
image = cv2.imread('fig1.jpg')

# 提取绿色的主要色块并获取平滑的轮廓
green_blocks = extract_green_color_blocks(image)

# 显示提取出的绿色色块及其轮廓
cv2.imshow('Green Color Blocks with Contours', green_blocks)
cv2.waitKey(0)
cv2.destroyAllWindows()